# 2. Update data daily

In [1]:
import pickle
import pandas as pd
import hopsworks
import os

# from keys import TOMTOM_API_KEY, TOMTOM_API_KEY2, TOMTOM_API_KEY3
TOMTOM_API_KEY = os.getenv('TOMTOM_API_KEY')
TOMTOM_API_KEY2 = os.getenv('TOMTOM_API_KEY2')
TOMTOM_API_KEY3 = os.getenv('TOMTOM_API_KEY3')

from functions.TomTomAPI import get_traffic_map_from_grid
from functions.HolidaysnWeather import get_weather, is_holiday

/Users/diogosilva/Documents/Scalable ML and DeepL/Project-ID2223_TrafficApp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
coordinates = 59.34318, 18.05141 # Stockholm near Odenplan
zoom = 20

TOMTOM_keys = [TOMTOM_API_KEY, TOMTOM_API_KEY2, TOMTOM_API_KEY3]

### 2.1. Collect traffic data

In [3]:
# Getting grid variable from memory
with open('variables/grid.pickle', 'rb') as file:
    # Deserialize and retrieve grid from pickle file
    grid = pickle.load(file)

In [4]:
# Collecting traffic flow from grid
success = False
i = 0
while i < len(TOMTOM_keys) and not success:
    try:
        traffic_map = get_traffic_map_from_grid(TOMTOM_keys[i],grid, zoom = zoom)
        success = True
    except:
        print('Failed with key:', i)
        i += 1

Failed with key: 0
num of requests: 298


In [5]:
from shapely.geometry import LineString
import datetime

traffic_df = pd.DataFrame(traffic_map2)
traffic_df['coordinates'] = traffic_df['coordinates'].apply(lambda x: LineString([(point['longitude'], point['latitude']) for point in x['coordinate']]))
traffic_df = traffic_df.drop_duplicates()
traffic_df['coordinates'] = traffic_df['coordinates'].apply(lambda x: str(x))

traffic_df['relativeSpeed'] = traffic_df.apply(lambda x: x['currentSpeed'] - x['freeFlowSpeed'], axis = 1)

today = datetime.datetime.now()
today = today.replace(second=0, microsecond=0, minute=0, hour=today.hour) + datetime.timedelta(hours=today.minute//30)
traffic_df['date']= [today for i in range(traffic_df.shape[0])]

traffic_df = traffic_df.drop(columns = ['@version'])

print(traffic_df.shape)
traffic_df.head()

(183, 6)


,,,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure
date,frc,coordinates,,,,,,
2024-12-17,FRC6,"LINESTRING (18.047361336193802 59.342010331133054, 18.047573230706064 59.34179577386307, 18.04913561745761 59.34021728308633, 18.049217424832563 59.34013278083443, 18.049815557443026 59.339518562397586, 18.04989736481798 59.3394354262025, 18.050928674184462 59.33839203896625, 18.05185805960835 59.33744924414018, 18.051886222803006 59.33741974932061, 18.05198948785008 59.33730442005571, 18.0520189921493 59.33727625024616, 18.05265199347693 59.336659329834355, 18.052741847478956 59.336566782256796, 18.05288936897483 59.33634150372781, 18.052956424200204 59.33624626218505, 18.05304761930674 59.33612019924765)",23,23,114,114,1.0,False


### 2.2. Collect weather data

In [6]:
weather_df = get_weather(coordinates)
weather_df

,temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
2024-12-17,3.876,-0.074,0.0,33.48,279.180298


### 2.3. Collect weather data

In [7]:
holiday_status = is_holiday()
weather_df['holiday_status'] = holiday_status
weather_df

,temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,holiday_status
2024-12-17,3.876,-0.074,0.0,33.48,279.180298,0


### 2.4. Joining data and uploading to Hopsworks

In [8]:
# Get the API key from GitHub Secrets
HOPSWORKS_API_KEY = os.getenv('HOPSWORKS_API_KEY')

# Get AQI API KEY from secrets of hopsworks
proj = hopsworks.login(project="ID2223LAB1KTH")
conn = hopsworks.connection(host="c.app.hopsworks.ai", project=proj, api_key_value=HOPSWORKS_API_KEY)
secrets = conn.get_secrets_api()

2024-12-17 15:21:21,078 INFO: Initializing external client
2024-12-17 15:21:21,080 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-17 15:21:23,645 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1170583
2024-12-17 15:21:24,813 INFO: Closing external client and cleaning up certificates.
2024-12-17 15:21:24,819 INFO: Initializing external client
2024-12-17 15:21:24,821 INFO: Base URL: https://c.app.hopsworks.ai:443


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
# Initialize the feature store
fs = proj.get_feature_store()

# Define and insert the Forecast Weather Feature Group
feature_group_name = "stockholm_traffic"
version = 1
fg = fs.get_feature_group(name=feature_group_name, version=version)
fg.insert(traffic_df)

# Define and insert the Air Quality Feature Group
feature_group_name = "stockholm_weather_holiday"
version = 1
fg = fs.get_feature_group(name=feature_group_name, version=version)
fg.insert(weather_df)